<a href="https://colab.research.google.com/github/MicheleS3/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.2.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.2.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3.2"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu 

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-02-21 19:15:00--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2022-02-21 19:15:00 (6.49 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Jewelry_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   50423057|R135Q3VZ4DQN5N|B00JWXFDMG|     657335467|Everbling Purple ...|         Jewelry|          5|            0|          0|   N|                Y|           Beauties!|so beautiful even...| 2015-08-31|
|         US|   11262325|R2N0QQ6R4T7YRY|B00W5T1H9W|      26030170|925 Sterling Silv...|         Jewelry|          5|    

In [5]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R135Q3VZ4DQN5N|          5|            0|          0|   N|                Y|
|R2N0QQ6R4T7YRY|          5|            0|          0|   N|                N|
|R3N5JE5Y4T6W5M|          5|            0|          0|   N|                Y|
|R2I150CX5IVY9Q|          5|            0|          0|   N|                Y|
|R1RM9ICOOA9MQ3|          5|            0|          0|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [6]:
# Convert star_rating column to integer
from pyspark.sql.types import IntegerType
vine_df = vine_df.withColumn("star_rating",vine_df.star_rating.cast(IntegerType()))
vine_df.dtypes

[('review_id', 'string'),
 ('star_rating', 'int'),
 ('helpful_votes', 'int'),
 ('total_votes', 'int'),
 ('vine', 'string'),
 ('verified_purchase', 'string')]

In [22]:
# Filter the data and create a new DataFrame or table to retrieve all the rows where the total_votes count is equal to or greater than 20
# to pick reviews that are more likely to be helpful and to avoid having division by zero issues later on
helpful_votes_df = vine_df.filter("total_votes>=20")
helpful_votes_df.show(10)
helpful_votes_df.count()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R16YC6SMGKA8SR|          5|           23|         23|   N|                Y|
| R5O4WE9UM60B0|          1|           29|         29|   N|                Y|
|R2EKRVCRC7U0IY|          1|           22|         24|   N|                Y|
|R2OKV47GETH0L7|          5|           24|         24|   N|                Y|
| R36LII9IITE17|          1|           22|         24|   N|                Y|
|R35VKNE16PFY0H|          5|           22|         24|   N|                N|
|R3TG008LHHZZIN|          5|          123|        125|   N|                Y|
|R2FYQBKCC1XG4M|          5|           20|         20|   N|                Y|
| R63PV336NI5X7|          1|           27|         28|   N|                Y|
|R1R42WPHB5ZSWI|          5|           51|         52|   N|     

7962

In [23]:
# Filter the new helpful_votes_df and create a new DataFrame or table to retrieve all the rows where 
# the number of helpful_votes divided by total_votes is equal or greater than 50%
helpful_reviews_df = helpful_votes_df.filter((helpful_votes_df.helpful_votes/helpful_votes_df.total_votes) >= 0.5)
helpful_reviews_df.show(10)
helpful_reviews_df.count()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R16YC6SMGKA8SR|          5|           23|         23|   N|                Y|
| R5O4WE9UM60B0|          1|           29|         29|   N|                Y|
|R2EKRVCRC7U0IY|          1|           22|         24|   N|                Y|
|R2OKV47GETH0L7|          5|           24|         24|   N|                Y|
| R36LII9IITE17|          1|           22|         24|   N|                Y|
|R35VKNE16PFY0H|          5|           22|         24|   N|                N|
|R3TG008LHHZZIN|          5|          123|        125|   N|                Y|
|R2FYQBKCC1XG4M|          5|           20|         20|   N|                Y|
| R63PV336NI5X7|          1|           27|         28|   N|                Y|
|R1R42WPHB5ZSWI|          5|           51|         52|   N|     

7710

In [24]:
# Filter the helpful_reviews_df and create a new dataframe or table that retrieves
# all the rows where a review was written as part of the Vine program (paid), vine == 'Y'
paid_reviews_df = helpful_reviews_df.filter("vine == 'Y'")
paid_reviews_df.show(10)
paid_reviews_df.count()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R15PXA9XU9ZKSU|          4|           29|         29|   Y|                N|
|R2LJ4QYQ4PYDEH|          5|           24|         26|   Y|                N|
|R2PJWWQZ37WCIC|          5|           95|         96|   Y|                N|
|R33G2VDKRN2USY|          5|           21|         21|   Y|                N|
|R13R4X37T2U50A|          4|           52|         57|   Y|                N|
|R1X1J2NAQ39DF6|          4|           29|         34|   Y|                N|
|R2UV5XHFNMPXOX|          5|           60|         61|   Y|                N|
|R31V08BCC9X3MF|          5|           18|         22|   Y|                N|
|R2G00QWTYW9N2G|          5|           86|         91|   Y|                N|
| RL8H3NKKP989J|          5|           30|         34|   Y|     

21

In [25]:
# Filter the helpful_reviews_df and create a new dataframe or table that retrieves
# all the rows where a review was written as part of the Vine program (unpaid), vine == 'N'
unpaid_reviews_df = helpful_reviews_df.filter("vine == 'N'")
unpaid_reviews_df.show(10)
unpaid_reviews_df.count()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R16YC6SMGKA8SR|          5|           23|         23|   N|                Y|
| R5O4WE9UM60B0|          1|           29|         29|   N|                Y|
|R2EKRVCRC7U0IY|          1|           22|         24|   N|                Y|
|R2OKV47GETH0L7|          5|           24|         24|   N|                Y|
| R36LII9IITE17|          1|           22|         24|   N|                Y|
|R35VKNE16PFY0H|          5|           22|         24|   N|                N|
|R3TG008LHHZZIN|          5|          123|        125|   N|                Y|
|R2FYQBKCC1XG4M|          5|           20|         20|   N|                Y|
| R63PV336NI5X7|          1|           27|         28|   N|                Y|
|R1R42WPHB5ZSWI|          5|           51|         52|   N|     

7689

In [12]:
# Determine the total number of reviews
helpful_reviews_df.count()

7710

In [26]:
# Determine the total number of reviews paid
paid_reviews_count = paid_reviews_df.count()
paid_reviews_count

21

In [19]:
# Determine the total number of reviews unpaid
unpaid_reviews_count = unpaid_reviews_df.count()
unpaid_reviews_count

7689

In [27]:
# Determine the total number of 5 star reviews paid
paid_five_star_reviews = paid_reviews_df.filter(paid_reviews_df.star_rating==5).count()
paid_five_star_reviews

11

In [21]:
# Determine the total number of 5 star reviews unpaid
unpaid_five_star_reviews = unpaid_reviews_df.filter(unpaid_reviews_df.star_rating==5).count()
unpaid_five_star_reviews

4444

In [28]:
# Determine the percentage of total number of 5 star reviews paid
paid_five_star_percentage = (paid_five_star_reviews/paid_reviews_count) * 100
paid_five_star_percentage

52.38095238095239

In [29]:
# Determine the percentage of total number of 5 star reviews unpaid
unpaid_five_star_percentage = (unpaid_five_star_reviews/unpaid_reviews_count) * 100
unpaid_five_star_percentage

57.79685264663805